In [1]:
import sys
sys.path.append('../src')
# from dataloader import create_dataloaders
from dataloader_ER import create_dataloaders,create_dataloaders_multi_label
# from lora_model import LORAEngine,LORAEngineDeberta,LORAEngineDebertaMultiClass
from lora_model_vmap import LORAEngineDebertaMultiClass,LORAEngineDebertaMultiLabel

# from influence import IFEngine
# from influence_hyperinf import IFEngine
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# model_name_or_path="roberta-large"
# model_name_or_path="/home/yanmy/roberta-base"
# model_name_or_path="../../model/deberta-v3-base"
model_name_or_path="../../model/ModernBERT-base/"
# model_name_or_path="../../model/Qwen2.5-0.5B-Instruct/"
# model_name_or_path="../../model/roberta-large"
# task="mrpc"
task = "Router"
noise_ratio=0
batch_size=16
# target_modules=["query_proj", "key_proj", "value_proj"] ## deberta, for roberta, use ["value"]
# target_modules = 
target_modules = ['Wqkv', 'Wo', 'Wi']
# target_modules=["value_proj"] ## deberta, for roberta, use ["value"]
# target_modules=["q_proj"] ## deberta, for roberta, use ["value"]

device="cuda:7"
num_epochs=1
lr=3e-4


In [3]:

dataloader_outputs = create_dataloaders_multi_label(model_name_or_path=model_name_or_path,
                                        #    task=task,
                                        #    noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                           train_file = '/data/home/wangys/MoE-Example/Router/train.csv',
                                           valid_file = '/data/home/wangys/MoE-Example/Router/valid.csv',
                                           test_file = '/data/home/wangys/MoE-Example/Router/test.csv',
                                           max_length=2048)
# train_dataloader, eval_dataloader, noise_index, tokenized_datasets, collate_fn=dataloader_outputs
train_dataloader, eval_dataloader,test_dataloader, tokenized_datasets, collate_fn, label2id, id2label =dataloader_outputs

num_labels = len(label2id)



Dataset({
    features: ['text', 'labels'],
    num_rows: 22592
})


Map:   0%|          | 0/22592 [00:00<?, ? examples/s]

Map:   0%|          | 0/2824 [00:00<?, ? examples/s]

Map:   0%|          | 0/2824 [00:00<?, ? examples/s]

In [1]:
from datasets import load_dataset

# Dataset id from huggingface.co/dataset
dataset_id = "../datasets/LLM_Router"

# Load raw dataset
raw_dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(raw_dataset['train'])}")
print(f"Test dataset size: {len(raw_dataset['test'])}")

Train dataset size: 15306
Test dataset size: 4921


In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers import AutoModelForSequenceClassification,AutoTokenizer

from datasets import load_dataset

# Dataset id from huggingface.co/dataset
dataset_id = "../datasets/LLM_Router"

# Load raw dataset
raw_dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(raw_dataset['train'])}")
print(f"Test dataset size: {len(raw_dataset['test'])}")
 
# Model id to load the tokenizer
model_id = "../../model/ModernBERT-base/"
# model_id = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length = 1024 # set model_max_length to 512 as prompts are not longer than 1024 tokens

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['prompt'], padding='max_length', truncation=True, return_tensors="pt")

# Tokenize dataset
raw_dataset =  raw_dataset.rename_column("label", "labels") # to match Trainer
tokenized_dataset = raw_dataset.map(tokenize, batched=True,remove_columns=["prompt"])
# labels = tokenized_dataset["train"].features["labels"].names
# num_labels = len(labels)
# label2id, id2label = dict(), dict()
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label
# Prepare model labels - useful for inference
# labels = tokenized_dataset["train"].features["labels"].names
# num_labels = len(labels)
# label2id, id2label = dict(), dict()
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label
 
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=2
)

Train dataset size: 15306
Test dataset size: 4921


You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in ModernBertForSequenceClassification is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at ../../model/ModernBERT-base/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset, DatasetDict

# 设置CUDA设备
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Dataset folder containing train.csv, valid.csv, and test.csv
dataset_folder = "/data/home/wangys/MoE-Example/Router/"

# 加载 CSV 文件并创建 DatasetDict
def load_csv_dataset(folder_path):
    data_files = {
        "train": os.path.join(folder_path, "train.csv"),
        "valid": os.path.join(folder_path, "valid.csv"),
        "test": os.path.join(folder_path, "test.csv"),
    }

    dataset = {}
    for split, file_path in data_files.items():
        df = pd.read_csv(file_path,index_col=0).sample(n=2000).reset_index(drop=True)
        df.columns = ['text','labels']
        # 将 `labels` 列转换为 Python 列表对象
        df["labels"] = df["labels"].apply(eval)  # 使用 eval() 将字符串解析为列表
        dataset[split] = Dataset.from_pandas(df)

    return DatasetDict(dataset)

raw_dataset = load_csv_dataset(dataset_folder)

print(f"Train dataset size: {len(raw_dataset['train'])}")
print(f"Valid dataset size: {len(raw_dataset['valid'])}")
print(f"Test dataset size: {len(raw_dataset['test'])}")

# Model id to load the tokenizer
model_id = "../../model/ModernBERT-base/"
# model_id = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.model_max_length = 1024  # 设置最大长度

# 获取所有的标签列表
def extract_all_labels(dataset):
    all_labels = set()
    for split in dataset:  # 遍历 train/valid/test
        for labels in dataset[split]["labels"]:
            all_labels.update(labels)
    return sorted(all_labels)

all_labels = extract_all_labels(raw_dataset)
num_labels = len(all_labels)
label2id = {label: i for i, label in enumerate(all_labels)}
id2label = {i: label for label, i in label2id.items()}

print(f"Number of labels: {num_labels}")
print("Label2ID mapping:", label2id)

# Tokenize helper function
def tokenize(batch):
    # Tokenize the text column
    encodings = tokenizer(
        batch['text'],  # Handle a list of text inputs
        padding="max_length",
        truncation=True,
        max_length=tokenizer.model_max_length,
    )

    # Ensure labels are converted correctly
    if isinstance(batch['labels'][0], str):  # If labels are strings, evaluate them to convert to lists
        labels = [eval(label) for label in batch['labels']]
    else:  # Otherwise, assume they are already lists
        labels = [label2id[b] for b in batch['labels']]
    print(batch['labels'])
    # Convert labels to tensors
    labels_tensor = torch.tensor(labels, dtype=torch.float)
    
    # Combine tokenized inputs and labels
    encodings["labels"] = labels_tensor
    return encodings


# Tokenize dataset
tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=["text", "labels"])

# 准备模型
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, id2label=id2label, label2id=label2id
)

print("Model and dataset are ready for multi-label classification.")

In [9]:
import numpy as np
from sklearn.metrics import f1_score
 
# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(
            labels, predictions, labels=labels, pos_label=1, average="weighted"
        )
    return {"f1": float(score) if score == 1 else score}

In [3]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments
 
# Define training args
training_args = TrainingArguments(
    output_dir= "modernbert-llm-router",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
	num_train_epochs=5,
    bf16=True, # bfloat16 training 
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="none",
)
 
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
)

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

In [10]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = (predictions > 0.5).astype(int)  # Convert logits to binary predictions
    f1 = f1_score(labels, predictions, average="weighted")
    precision = precision_score(labels, predictions, average="weighted")
    recall = recall_score(labels, predictions, average="weighted")
    return {
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Define training args
training_args = TrainingArguments(
    output_dir="modernbert-llm-router",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=2,
    bf16=True,  # bfloat16 training 
    optim="adamw_torch_fused",  # improved optimizer 
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    # push to hub parameters
    report_to="none",
    no_cuda=True
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
)

/home/wangys/anaconda3/envs/bert24/lib/python3.11/site-packages/transformers/training_args.py:1588: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [4]:


lora_engine = LORAEngineDebertaMultiLabel(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            # eval_dataloader=eval_dataloader,
                            eval_dataloader=eval_dataloader,
                            test_dataloader = test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            low_rank=16, 
                            task=task,
                            save_path = '../models/ER',
                            valid_each_epoch=True,
                            cal_grad_per_sample = False,
                            num_labels=num_labels,
                            label2id=label2id,
                            id2label=id2label,
                            use_lora = True
                            )

In [5]:
lora_engine.build_LORA_model()
all_logits = lora_engine.train_LORA_model()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at ../../model/ModernBERT-base/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,383,814 || all params: 152,993,292 || trainable%: 2.2117
Total Steps:1412


100%|██████████| 1412/1412 [04:54<00:00,  4.80it/s]


Epoch 1: Training Loss = 0.32238240604402324
Epoch 1:Precision: 0.8750, Recall: 1.0000, F1-score: 0.9333


100%|██████████| 177/177 [00:14<00:00, 12.37it/s]

Precision: 0.9012, Recall: 1.0000, F1-score: 0.9480
